# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `UBC-NLP/AraT5-base` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/arabic_dialogue_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [ ]:
!pip install datasets

In [24]:
from datasets import load_dataset

Data = load_dataset("CUTD/arabic_dialogue_df",split = 'train[:10%]')
Data = Data.train_test_split(test_size = 0.2)

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/AraT5-base")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [4]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [28]:
tokenized_Data = Data.map(preprocess_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [26]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="UBC-NLP/AraT5-base" , return_tensors="tf")

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [7]:
from transformers import TFAutoModelForSeq2SeqLM
model = TFAutoModelForSeq2SeqLM.from_pretrained("UBC-NLP/AraT5-base")

tf_model.h5:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at UBC-NLP/AraT5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [8]:
tokenized_Data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 120
    })
    test: Dataset({
        features: ['Unnamed: 0', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 30
    })
})

In [9]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [29]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_Data["train"],
    batch_size=4,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_Data["test"],
    batch_size=4,
    collate_fn=data_collator,
)

## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [11]:
import tensorflow as tf

model.compile(optimizer=optimizer)

## Step 8: Fine-tune the Model

In [30]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=2,  batch_size=4)

Epoch 1/2
300/300 [==============================] - 160s 533ms/step - loss: 21.3601 - val_loss: 18.6801
Epoch 2/2
300/300 [==============================] - 159s 529ms/step - loss: 18.0956 - val_loss: 15.9328


Train the model using the specified arguments and dataset.

## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [34]:
Data['test'][0]

{'Unnamed: 0': 1495,
 'dialogue': 'الشخص 1: توم ، أنت صديقي العزيز ، أليس كذلك؟\n  الشخص 2: بالتأكيد.ما خطبك؟\n  الشخص 1: الامتحان!الامتحان الرهيب!أنا لست مستعدة بعد.هل يمكن أن تخبرني كيف أحصل على علامات جيدة؟\n  الشخص 2: لا تكون متوترة للغاية!يستريح!أولا ، النوم في وقت مبكر من الليل.ثانياً ، دراسة ساعتين في اليوم بعد المدرسة.ثالثًا ، لا تتعب نفسك ، لكن لا تكون كسولًا.هل سيساعد ذلك؟\n  الشخص 1: نعم ، شكرا لك.',
 'summary': 'يسأل الشخص 1 توم كيفية الحصول على علامات جيدة في الامتحان.'}

In [36]:
text = Data['test'][0]['dialogue']

In [37]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model,tokenizer=tokenizer )
summarizer(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'summary_text': '🚫 شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة شنطة'}]